In [4]:
from bs4 import BeautifulSoup
from twilio.rest import Client
from datetime import datetime
import pandas as pd
import requests

# Application #1: Upcoming Earnings Text Updates

This Notebook contains a script that scans for company's that are releasing their earnings today. It will return a dataframe containing the tickers that are releasing their earnings. It will then send a text message to my mobile using Twilio with the formatted information regarding which companies are reporting today. 

This list of tickers will then subsequently be used to check if any of the stocks are currently consolidating using our function from the "Signals" notebook. 

### Upcoming Earnings for all US Companies

In [2]:
# initialising empty dataframe which will hold all tickers
df = pd.DataFrame()

# defining headers to simulate a browser call (helps get past some websites' defences against scraping)
headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
    'Cache-Control': 'max-age=0',
    'Connection': 'close',
    'DNT': '1', # Do Not Track Request Header 
    'Pragma': 'no-cache',
    'Referrer': 'https://google.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36'
}

for i in range(1, 5):
    
    # the URL which updates on each loop iteration
    url = "https://seekingalpha.com/earnings/earnings-calendar/{}".format(i)

    # Make a GET request to fetch the raw HTML content
    html_content = requests.get(url, headers=headers).text

    # get the HTML content
    soup = BeautifulSoup(html_content, "lxml")

    # obtaining tables
    table = soup.find_all('table')
    
    # indexing the relevant tables to obtain data of interest
    data = pd.read_html(str(table))[0].set_index("Release Date").iloc[:,:-1]
    
    df = df.append(data)
    
df.head(5)

,Symbol,Name,Release Time
Release Date,,,
10/01/2021,KFMNF,Kaufman & Broad S.A.,Post-Market
10/01/2021,JDWPF,J D Wetherspoon plc,Estimated
10/01/2021,DSKIF,"Daiseki Co.,Ltd.",Estimated
10/01/2021,ZOJIF,Zojirushi Corporation,Estimated
10/01/2021,PSLKF,Phoslock Environmental Technologies Limited,Estimated


### Formatting the data to send a text message

In [3]:
df.index = pd.to_datetime(df.index)

today = datetime.today().strftime('%Y-%m-%d')

todays_companys = df[df.index == today]

info = []

for i in range(len(todays_companys)):
    todays_companys.iloc[i, :]
    
    string = "{} ({}) is reporting today. The release time is {}.".format(todays_companys.iloc[i, :]["Name"], todays_companys.iloc[i, :]["Symbol"], todays_companys.iloc[i, :]["Release Time"])
    
    info.append(string)
    
msg = "\n".join(info)

### Sending a text with Twilio

In [ ]:
# the following line needs your Twilio Account SID and Auth Token
client = Client("ACe737e611e4524cdf0ed7feb19d4ef564", "044ca567300180705e108e76715dd60e")

# change the "from_" number to your Twilio number and the "to" number
# to the phone number you signed up for Twilio with, or upgrade your
# account to send SMS to any phone number
client.messages.create(to="+447719104171", from_="+447897033826", body=msg)